In [ ]:
import requests
import re
import os
import time

In [ ]:
# judge if the folder exist. 判断图集的文件夹是否存在，如果不存在则建立一个
def fJudgeFileNetNGlyc(vSeqName):    
    global vFoldNameNetNGlyc
    vFoldNameNetNGlyc = "E:\\Store\\code\\02.Python\\03.Request\\Bio" + "\\" + vSeqName + "\\NetNGlyc糖基化位点\\"
    os.makedirs(vFoldNameNetNGlyc, exist_ok = True)
    os.chdir(vFoldNameNetNGlyc)
    os.makedirs("Pic", exist_ok = True)

In [ ]:
def fGetNetNGlyc(vSeq):
    vUrl = "http://www.cbs.dtu.dk/cgi-bin/webface2.fcgi"
    vPostData ={"configfile": "/usr/opt/www/pub/CBS/services/NetNGlyc-1.0/NetNGlyc.cf",
                "SEQPASTE": vSeq,                
                "graphics": "ps"}
    r = requests.post(vUrl, data = vPostData)
    time.sleep(5)
    vRealUrl = re.search("a href=\"([\s\S]*?)\"", r.text, re.S).group(1)
    time.sleep(5)
    r = requests.get(vRealUrl)
    return(r.text)

In [ ]:
def fSaveNetNGlycPic(vHtml):
    global vCode    
    vCode = str(re.search("http://www.cbs.dtu.dk//services/NetNGlyc/tmp//(.*?).gif", vHtml, re.S).group(1))    
    vPicA = "http://www.cbs.dtu.dk//services/NetNGlyc/tmp//" + vCode + ".gif"    
    vPicB = "http://www.cbs.dtu.dk//services/NetNGlyc/tmp//" + vCode + ".eps"    
    with open("Pic\\NetNGlyc.gif", "wb") as vPic1:
        r = requests.get(vPicA)
        vPic1.write(r.content)
    with open("Pic\\NetNGlyc.eps", "wb") as vPic2:
        r = requests.get(vPicB)
        vPic2.write(r.content)   

In [ ]:
def fSoupNetNGlyc(vHtml):
    vNetNGlycText = vHtml
    vNetNGlyc1 = vNetNGlycText.replace(re.search("[\s\S]*?\</h2\>", vNetNGlycText, re.S).group(0), "<h1>NetNGlyc糖基化位点分析</h1><p><a href = \"http://www.cbs.dtu.dk/services/NetNGlyc/\">使用NetNGlyc 1.0查询</a></p>")
    vNetNGlyc2 = vNetNGlyc1.replace(re.search("\<b\>Graphics[\s\S]*", vNetNGlyc1, re.S).group(0), "</a><h3>文件下载</h3><p><a href=\"file:///" + vFoldNameNetNGlyc + "Pic\\NetNGlyc.gif\">gif图片</a><p><a href=\"file:///" + vFoldNameNetNGlyc + "Pic\\NetNGlyc.eps\">eps图片</a></p>")
    vNetNGlyc3 = vNetNGlyc2.replace("     Asn-Xaa-Ser/Thr sequons in the sequence output below are highlighted in <font color=\"#0000ff\"><b>blue</b></font>", "Asn Xaa Ser/Thr使用高亮的<font color=\"#0000ff\"><b>蓝色</b></font>")
    vNetNGlyc4 = vNetNGlyc3.replace("http://www.cbs.dtu.dk//services/NetNGlyc/tmp//" + vCode + ".gif", "file:///" + vFoldNameNetNGlyc + "\Pic" + "\\NetNGlyc.gif")
    vNetNGlyc5 = vNetNGlyc4.replace("http://www.cbs.dtu.dk//services/NetNGlyc/tmp//" + vCode + ".eps", "file:///" + vFoldNameNetNGlyc + "\Pic" + "\\NetNGlyc.eps")
    vNetNGlyc6 = vNetNGlyc5.replace("          Asparagines predicted to be N-glycosylated are highlighted in <font color=\"#ff0000\"><b>red</font>", "N-糖基化的天冬酰胺使用高亮的<font color=\"#ff0000\"><b>红色</font>")
    vNetNGlyc7 = vNetNGlyc6.replace("Output for 'Sequence'", "预测结果").replace("Position", "位置").replace("Potential", "可能性").replace("Technical University of Denmark", "丹麦科技大学")
    vNetNGlyc8 = vNetNGlyc7.replace("You can get the prediction graphics shown above in one of", "")
    vNetNGlyc9 = vNetNGlyc8.replace("", "").replace("", "")
    vNetNGlyc10 = vNetNGlyc9.replace("", "").replace("", "").replace("", "").replace("", "")
    vNetNGlyc = vNetNGlyc10
    with open("NetNGlyc.html", "w") as f:
        f.write(vNetNGlyc)

In [ ]:
def main(vSeqName, vSeq):
    print("===开始NetNGlyc糖基化位点分析===")
    fJudgeFileNetNGlyc(vSeqName)
    print("---------已创建存储目录---------")
    print("----------开始提交数据----------")
    vHtml = fGetNetNGlyc(vSeq)
    print("---------已获取分析结果---------")
    print("----------开始保存数据----------")
    fSaveNetNGlycPic(vHtml)
    print("----------开始保存报告----------")
    fSoupNetNGlyc(vHtml)
    print("---------结果已保存完毕---------")

In [ ]:
with open("name.txt", "r") as f:
    vSeqName = f.read()
with open("orf.txt", "r") as f:
    vSeq = f.read()
main(vSeqName, vSeq)